<a href="https://colab.research.google.com/github/AhmedAlYousif-AI/O0Zdp1U9TiVA1Qvv/blob/main/Potential%20Talents%20Ranking%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing the Essential libraries and their Functions

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk
import re

In [2]:
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [3]:
data = pd.read_csv("/content/potential-talents.csv")

data

,id,job_title,location,connection,fit
0,1.0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2.0,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3.0,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4.0,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5.0,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
994,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN


**Feature Engineering**

EDA

In [4]:
data.head()

,id,job_title,location,connection,fit
0,1.0,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2.0,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3.0,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4.0,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5.0,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
data.tail()

,id,job_title,location,connection,fit
994,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    float64
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 39.2+ KB


In [7]:
data.isnull().sum()

,0
id,895
job_title,895
location,895
connection,895
fit,999


In [8]:
data.describe()

,id,fit
count,104.000000,0.0
mean,52.500000,NaN
std,30.166206,NaN
min,1.000000,NaN
25%,26.750000,NaN
50%,52.500000,NaN
75%,78.250000,NaN
max,104.000000,NaN


In [9]:
data.count()

,0
id,104
job_title,104
location,104
connection,104
fit,0


In [10]:
data.dtypes.value_counts()

,count
object,3
float64,2


In [11]:
data.drop(['id', 'location', 'fit'], axis=1, inplace=True)

data

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500+
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500+
4,Advisory Board Member at Celal Bayar University,500+
...,...,...
994,NaN,NaN
995,NaN,NaN
996,NaN,NaN
997,NaN,NaN


In [12]:
data_2 = data.copy()

data_2

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500+
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500+
4,Advisory Board Member at Celal Bayar University,500+
...,...,...
994,NaN,NaN
995,NaN,NaN
996,NaN,NaN
997,NaN,NaN


In [13]:
data_2.dropna(subset=['job_title'], inplace=True)

data_2

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500+
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500+
4,Advisory Board Member at Celal Bayar University,500+
...,...,...
99,Aspiring Human Resources Manager | Graduating ...,103
100,Human Resources Generalist at Loparex,500+
101,Business Intelligence and Analytics at Travelers,49
102,Always set them up for Success,500+


In [14]:
data_2.isnull().sum()

,0
job_title,0
connection,0


In [15]:
data_2.head()

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500+
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500+
4,Advisory Board Member at Celal Bayar University,500+


In [16]:
data_2.tail()

,job_title,connection
99,Aspiring Human Resources Manager | Graduating ...,103
100,Human Resources Generalist at Loparex,500+
101,Business Intelligence and Analytics at Travelers,49
102,Always set them up for Success,500+
103,Director Of Administration at Excellence Logging,500+


**Word Embedding**

In [17]:
pip install sentence_transformers

Bert

In [18]:
from sentence_transformers import SentenceTransformer

data_2['job_title'] = data_2['job_title'].astype(str)

# Load Pretrained MiniLM Model
model = SentenceTransformer('bert-base-uncased')  # ---D embeddings

# Generate Sentence Embeddings
data_2['job_title_embedding'] = data_2['job_title'].apply(lambda x: model.encode(x))

# Display Results
print(data_2[['job_title', 'job_title_embedding']])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


                                             job_title  \
0    2019 C.T. Bauer College of Business Graduate (...   
1    Native English Teacher at EPIK (English Progra...   
2                Aspiring Human Resources Professional   
3               People Development Coordinator at Ryan   
4      Advisory Board Member at Celal Bayar University   
..                                                 ...   
99   Aspiring Human Resources Manager | Graduating ...   
100              Human Resources Generalist at Loparex   
101   Business Intelligence and Analytics at Travelers   
102                     Always set them up for Success   
103   Director Of Administration at Excellence Logging   

                                   job_title_embedding  
0    [-0.4133859, 0.079430476, 0.1310354, -0.063541...  
1    [-0.4732027, -0.048262026, 0.05471474, -0.3529...  
2    [-0.017994875, -0.12040844, -0.052445665, 0.01...  
3    [-0.35929012, -0.14920664, -0.2183593, -0.0064...  
4    [-0.48415574,

In [19]:
w1 = model.encode('Aspiring human resources')

In [20]:
data_2['Bert_cosine_similarity'] = data_2['job_title_embedding'].apply(lambda x: cosine_similarity([w1], [x])[0][0])

data_2['Bert_cosine_similarity']

,Bert_cosine_similarity
0,0.587713
1,0.546800
2,0.902632
3,0.752288
4,0.430944
...,...
99,0.659241
100,0.681012
101,0.645834
102,0.621722


Mini_LM

In [21]:
from sentence_transformers import SentenceTransformer

data_2['job_title'] = data_2['job_title'].astype(str)

# Load Pretrained MiniLM Model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # 384D embeddings

# Generate Sentence Embeddings
data_2['job_title_embedding'] = data_2['job_title'].apply(lambda x: model.encode(x))

# Display Results
print(data_2[['job_title', 'job_title_embedding']])

                                             job_title  \
0    2019 C.T. Bauer College of Business Graduate (...   
1    Native English Teacher at EPIK (English Progra...   
2                Aspiring Human Resources Professional   
3               People Development Coordinator at Ryan   
4      Advisory Board Member at Celal Bayar University   
..                                                 ...   
99   Aspiring Human Resources Manager | Graduating ...   
100              Human Resources Generalist at Loparex   
101   Business Intelligence and Analytics at Travelers   
102                     Always set them up for Success   
103   Director Of Administration at Excellence Logging   

                                   job_title_embedding  
0    [-0.07176389, 0.0022636761, -0.017858269, 0.03...  
1    [-0.012136351, 0.01905448, 0.03814818, 0.01352...  
2    [-0.031211179, 0.047989447, -0.014072431, 0.09...  
3    [-0.106952325, -0.02074511, -0.08798131, 0.041...  
4    [-0.09358538,

In [22]:
w1 = model.encode('Aspiring human resources')

In [23]:
data_2['Mini-LM_cosine_similarity'] = data_2['job_title_embedding'].apply(lambda x: cosine_similarity([w1], [x])[0][0])

data_2['Mini-LM_cosine_similarity']

,Mini-LM_cosine_similarity
0,0.573268
1,0.239483
2,0.949807
3,0.380222
4,0.230512
...,...
99,0.760215
100,0.603002
101,0.086457
102,0.148272


FastText

In [24]:
!pip install fasttext

In [25]:
from huggingface_hub import hf_hub_download
import fasttext
nltk.download('punkt_tab')

fasttext_model = fasttext.load_model(hf_hub_download("facebook/fasttext-en-vectors", "model.bin"))

# Function to clean and tokenize text
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenization
    return [word for word in tokens if word in fasttext_model]  # Keep only known words

# Function to get sentence vector by averaging word vectors
def sentence_to_vector(sentence, model):
    words = preprocess_text(sentence)
    if not words:
        return np.zeros(model.vector_size)  # Return zero vector if no valid words
    return np.mean([model[word] for word in words], axis=0)

# Apply FastText to each sentence in the column
data_2['job_title_embedding'] = data_2['job_title'].apply(lambda x: sentence_to_vector(x, fasttext_model))

# Display results
print(data_2[['job_title', 'job_title_embedding']])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                             job_title  \
0    2019 C.T. Bauer College of Business Graduate (...   
1    Native English Teacher at EPIK (English Progra...   
2                Aspiring Human Resources Professional   
3               People Development Coordinator at Ryan   
4      Advisory Board Member at Celal Bayar University   
..                                                 ...   
99   Aspiring Human Resources Manager | Graduating ...   
100              Human Resources Generalist at Loparex   
101   Business Intelligence and Analytics at Travelers   
102                     Always set them up for Success   
103   Director Of Administration at Excellence Logging   

                                   job_title_embedding  
0    [0.0152725605, -0.022511747, -0.024433596, 0.0...  
1    [0.024544511, 0.0061443807, -0.02365049, -0.00...  
2    [0.017723147, -0.017749855, -0.019734647, 0.08...  
3    [0.042334706, 0.05361083, 0.033050507, -0.0071...  
4    [0.06376933, 

In [26]:
w1 = sentence_to_vector('Aspiring human resources', fasttext_model)

In [27]:
data_2['FastText_cosine_similarity'] = data_2['job_title_embedding'].apply(lambda x: cosine_similarity([w1], [x])[0][0])

data_2['FastText_cosine_similarity']

,FastText_cosine_similarity
0,0.395398
1,0.253072
2,0.982771
3,0.180921
4,0.164732
...,...
99,0.500876
100,0.410244
101,0.268428
102,0.257655


Labeling

In [28]:
data_2 = data_2[['job_title', 'job_title_embedding', 'Bert_cosine_similarity' ,'Mini-LM_cosine_similarity', 'FastText_cosine_similarity', 'connection']]

data_2

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,connection
0,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398,85
1,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072,500+
2,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,44
3,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921,500+
4,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732,500+
...,...,...,...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876,103
100,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244,500+
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,49
102,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655,500+


In [29]:
data_3 = data_2.copy()

Converting ('500+ ') to (500) for Connection Column

In [30]:
data_3['connection'] = data_2['connection'].replace(['500+ '], [500], inplace=True)

data_3['connection']

<ipython-input-30-6c6aa7968bff>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_3['connection'] = data_2['connection'].replace(['500+ '], [500], inplace=True)
<ipython-input-30-6c6aa7968bff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_3['connection'] = data_2['connection'].replace(['500+ '], [500], inplace=True)


,connection
0,None
1,None
2,None
3,None
4,None
...,...
99,None
100,None
101,None
102,None


In [31]:
data_2['connection'] = data_2['connection'].astype(int)

data_2['connection']

,connection
0,85
1,500
2,44
3,500
4,500
...,...
99,103
100,500
101,49
102,500


In [32]:
data_2['connection'].unique()

array([ 85, 500,  44,   1,  61,   2, 390,  57,  82,   5,   7,  16, 212,
       409,  52, 455, 174, 268,  50,   4,  40,  18, 349, 155,  39,  64,
         9, 415,  19,  71,  48, 103,  49])

In [33]:
data_2

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,connection
0,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398,85
1,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072,500
2,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,44
3,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921,500
4,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732,500
...,...,...,...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876,103
100,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244,500
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,49
102,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655,500


In [34]:
ranked_data = data_2.copy()

ranked_data

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,connection
0,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398,85
1,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072,500
2,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,44
3,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921,500
4,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732,500
...,...,...,...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876,103
100,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244,500
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,49
102,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655,500


Feature Scaling

In [35]:
ranked_data.drop(['job_title', 'job_title_embedding', 'Bert_cosine_similarity' ,'Mini-LM_cosine_similarity', 'FastText_cosine_similarity'], axis=1, inplace=True)

ranked_data

,connection
0,85
1,500
2,44
3,500
4,500
...,...
99,103
100,500
101,49
102,500


In [36]:
scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(ranked_data)
ranked_data['scaled_connection'] = pd.DataFrame(scaled_data, columns=ranked_data.columns)

ranked_data['scaled_connection']

,scaled_connection
0,0.168337
1,1.000000
2,0.086172
3,1.000000
4,1.000000
...,...
99,0.204409
100,1.000000
101,0.096192
102,1.000000


In [37]:
ranked_data

,connection,scaled_connection
0,85,0.168337
1,500,1.000000
2,44,0.086172
3,500,1.000000
4,500,1.000000
...,...,...
99,103,0.204409
100,500,1.000000
101,49,0.096192
102,500,1.000000


Labeling (2)

In [38]:
ranked_data[['job_title', 'job_title_embedding', 'Bert_cosine_similarity','Mini-LM_cosine_similarity', 'FastText_cosine_similarity']] = data_2[['job_title', 'job_title_embedding', 'Bert_cosine_similarity' ,'Mini-LM_cosine_similarity', 'FastText_cosine_similarity']]

ranked_data

,connection,scaled_connection,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity
0,85,0.168337,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398
1,500,1.000000,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072
2,44,0.086172,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771
3,500,1.000000,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921
4,500,1.000000,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732
...,...,...,...,...,...,...,...
99,103,0.204409,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876
100,500,1.000000,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244
101,49,0.096192,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428
102,500,1.000000,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655


In [39]:
ranked_data.drop(['connection'], axis=1, inplace=True)

ranked_data

,scaled_connection,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity
0,0.168337,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398
1,1.000000,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072
2,0.086172,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771
3,1.000000,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921
4,1.000000,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732
...,...,...,...,...,...,...
99,0.204409,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876
100,1.000000,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244
101,0.096192,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428
102,1.000000,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655


Labeling (3)

In [40]:
ranked_data = ranked_data[['job_title', 'job_title_embedding', 'Bert_cosine_similarity','Mini-LM_cosine_similarity', 'FastText_cosine_similarity', 'scaled_connection']]

ranked_data

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,scaled_connection
0,2019 C.T. Bauer College of Business Graduate (...,"[0.0152725605, -0.022511747, -0.024433596, 0.0...",0.587713,0.573268,0.395398,0.168337
1,Native English Teacher at EPIK (English Progra...,"[0.024544511, 0.0061443807, -0.02365049, -0.00...",0.546800,0.239483,0.253072,1.000000
2,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172
3,People Development Coordinator at Ryan,"[0.042334706, 0.05361083, 0.033050507, -0.0071...",0.752288,0.380222,0.180921,1.000000
4,Advisory Board Member at Celal Bayar University,"[0.06376933, 0.061760258, 0.0017473322, -0.007...",0.430944,0.230512,0.164732,1.000000
...,...,...,...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,"[0.0048759514, -0.0412865, -0.0027207024, 0.05...",0.659241,0.760215,0.500876,0.204409
100,Human Resources Generalist at Loparex,"[0.06540059, 0.079081535, 0.0039790524, 0.0283...",0.681012,0.603002,0.410244,1.000000
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,0.096192
102,Always set them up for Success,"[-0.0083640115, -0.17212616, 0.028542055, 0.15...",0.621722,0.148272,0.257655,1.000000


Ranking (Final Score)

In [45]:
ranked_data['ranking'] = 0.30 * ranked_data['Bert_cosine_similarity'] + 0.30 * ranked_data['Mini-LM_cosine_similarity'] + 0.30 * ranked_data['FastText_cosine_similarity'] + 0.10 * ranked_data['scaled_connection']

ranked_data['ranking']

,ranking
27,0.826483
29,0.826483
96,0.864591
45,0.859181
20,0.859181
...,...
53,0.324355
90,0.315230
101,0.309835
92,0.286918


In [46]:
ranked_data

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,scaled_connection,ranking
27,Seeking Human Resources Opportunities,"[0.008069926, -0.0018737718, 0.012377604, 0.08...",0.823453,0.799642,0.871997,0.779559,0.826483
29,Seeking Human Resources Opportunities,"[0.008069926, -0.0018737718, 0.012377604, 0.08...",0.823453,0.799642,0.871997,0.779559,0.826483
96,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.140281,0.864591
45,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
20,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
...,...,...,...,...,...,...,...
53,Student at Chapman University,"[0.06621926, 0.09548319, -0.021614028, -0.0231...",0.550157,0.351657,0.178703,0.002004,0.324355
90,Lead Official at Western Illinois University,"[0.022828253, 0.06405978, -0.012028996, 0.0021...",0.529710,0.275150,0.220523,0.076152,0.315230
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,0.096192,0.309835
92,Admissions Representative at Community medical...,"[0.05145613, 0.05081804, -0.01572166, 0.016796...",0.548684,0.161050,0.241315,0.016032,0.286918


Sorting (Descending)

In [47]:
ranked_data = ranked_data.sort_values(by='ranking', ascending=False)

ranked_data['ranking']

,ranking
96,0.864591
45,0.859181
20,0.859181
32,0.859181
16,0.859181
...,...
53,0.324355
90,0.315230
101,0.309835
92,0.286918


In [49]:
ranked_data

,job_title,job_title_embedding,Bert_cosine_similarity,Mini-LM_cosine_similarity,FastText_cosine_similarity,scaled_connection,ranking
96,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.140281,0.864591
45,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
20,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
32,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
16,Aspiring Human Resources Professional,"[0.017723147, -0.017749855, -0.019734647, 0.08...",0.902632,0.949807,0.982771,0.086172,0.859181
...,...,...,...,...,...,...,...
53,Student at Chapman University,"[0.06621926, 0.09548319, -0.021614028, -0.0231...",0.550157,0.351657,0.178703,0.002004,0.324355
90,Lead Official at Western Illinois University,"[0.022828253, 0.06405978, -0.012028996, 0.0021...",0.529710,0.275150,0.220523,0.076152,0.315230
101,Business Intelligence and Analytics at Travelers,"[0.021158433, 0.051734198, 0.0146321105, -0.01...",0.645834,0.086457,0.268428,0.096192,0.309835
92,Admissions Representative at Community medical...,"[0.05145613, 0.05081804, -0.01572166, 0.016796...",0.548684,0.161050,0.241315,0.016032,0.286918
